In [1]:
import numpy as np
import os,tqdm
import os
import sys
sys.path.append('/home/ash/deeplab/')
import numpy as np
from PIL import Image
from torch.utils import data
import torch
from torchvision import transforms
import math

def predict_third_point(x1,x2,theta):
    '''
    预测第三个点
    '''
    # np.cos输入是弧度制
    return x1*x2 / (2*x1*np.cos(theta) - x2)

def conpare_third_point(x,x_predict,threshod=0.4):
    # 区分左右跳变
    if (x-x_predict)>threshod:
        return 1
    elif (x_predict - x)>threshod:
        return -1
    else:
        return 0


# 2度以上认为是大障碍物
FOV_THRE = 2
FOV_THRE = np.deg2rad(FOV_THRE)
LIDAR_FOV_RESOLUSION = math.radians(0.2)

def find_obstacle_by_breakpoint(breakpoints, obstacle, FOV_threshold=FOV_THRE):
    FOV_threshold_pixel = round(FOV_threshold / LIDAR_FOV_RESOLUSION)
    """
    args：

        breakpoints 输入的一条id的lidar
        obstacle 输出的障碍物属性标签
        FOV_threshold_pixel 阈值，大于 2°/0.2°认为是大障碍物

    TODO 注意 ：

        角度遍历是逆时针的

        这里假设如下，有问题再翻转一下
            从-1到1，负障碍物
            从1到-1，正障碍物

    应该防止的BUG：

        连续出现三个及以上的+-1跳变，说明障碍物表面复杂，或者有噪声
    """
    # 跳变点状态
    positiveFlag = 0
    negativeFlag = 0
    # 当前状态索引
    positiveIndex = 0
    negativeIndex = 0
    # 计数器
    cnt = 0

    for i in breakpoints:
        cnt = cnt + 1
        if i == -1:
            negativeFlag = 1
            negativeIndex = cnt
        elif i == 1:
            positiveFlag = 1
            positiveIndex = cnt

        # 检查状态得在每轮结束时候，这样就能知道顺序了，减少一次比较
        # 一对点找到了

        if positiveFlag and negativeFlag:
            # print('状态',positiveFlag,negativeFlag,positiveIndex,negativeIndex)
            positiveFlag = 0
            negativeFlag = 0

            # print(math.fabs(positiveIndex-negativeIndex),FOV_threshold_pixel)
            if math.fabs(positiveIndex - negativeIndex) <= FOV_threshold_pixel:
                # 第二个是负，为正障碍物
                if i == -1:
                    print("正", positiveIndex, negativeIndex)
                    obstacle[positiveIndex:negativeIndex] = np.ones(negativeIndex - positiveIndex)
                # 第二个是正，为负障碍物
                if i == 1:
                    print("负", positiveIndex, negativeIndex)
                    obstacle[negativeIndex:positiveIndex] = -np.ones(positiveIndex - negativeIndex)


OSError: [WinError 182] 操作系统无法运行 %1。 Error loading "C:\ProgramData\Anaconda3\envs\t\lib\site-packages\torch\lib\caffe2_detectron_ops_gpu.dll" or one of its dependencies.

In [ ]:
velodyne = r'F:\Small_Obstacle_Dataset\train\file_2\velodyne\0000000007.npy'
odometry = r'F:\Small_Obstacle_Dataset\train\file_2\odometry\0000000007.npy'

f = np.load(velodyne,allow_pickle=True)

root_path = r'F:\Small_Obstacle_Dataset\train\file_2\velodyne'
folders=os.listdir(root_path)

imgs = []
for f in folders:
    #print(f)
    seg_path = os.path.join(root_path, f)
    print(seg_path)
    img = np.load(seg_path,allow_pickle=True)
    imgs.append(img)
